In [86]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [87]:
df = pd.read_excel('pooldata.xlsx')
meta = df.iloc[0, 1:]
df.columns = df.iloc[1]
df = df.iloc[2:]

df.head()

1,Date,Game,Break,Issac Shots,Anu Shots,Winner,Method,Balls Left,Notes,Issac Wins,Anu Wins
2,2025-05-20 00:00:00,678,Anagha,"0,1,0,0,0,1,0,0,1,1,0,2,0S,0,1,","0,0,0,0,1,0,3,0,1S,0S,0,0,0,1,0,B",Anagha,Normal,0,NaN,420,258
3,2025-05-20 00:00:00,677,Issac,"0,2,0,1,0S,0,0,0,1S,1,0,0,1,","0,0S,0,0,1,2,0,1S,2S,0,0,1S,B",Anagha,Normal,1,NaN,420,257
4,2025-05-20 00:00:00,676,Anagha,"0,0,3,0S,0,-1,1,1,0,2B","0,0,0,0,3,0,0,1,2,0S,",Issac,Normal,0,NaN,420,256
5,2025-05-20 00:00:00,675,Issac,"0,1,-1,0,1,4,1,0,0,","0,0,0,1,3S,0,1,0,1B",Anagha,Normal,0,NaN,419,256
6,2025-05-20 00:00:00,674,Anagha,"1,1,0,0,0,0,0,2-1,0,1,0,1-1,0,1B","0,0,0,0,1,2,0,0,0,0S,0,0,2,0,",Issac,Normal,0,NaN,419,255


In [88]:
df.iloc[:, 1:].describe()

1,Game,Break,Issac Shots,Anu Shots,Winner,Method,Balls Left,Notes,Issac Wins,Anu Wins
count,589,520,509,510,588,588,458,50,589,589
unique,589,2,507,509,2,4,8,47,366,224
top,90,Anagha,"0,1,","0,",Issac,Normal,1,Scratches not counted,56,200
freq,1,261,3,2,365,457,130,4,7,13


In [89]:
fig = px.area(df, x='Date', y='Game', title='Game Count by Date')
fig.show()


In [90]:
fig = px.area(df, x='Date', y=['Issac Wins', 'Anu Wins'], 
              title='Wins Over Time',
              labels={'value': 'Number of Wins', 'variable': 'Player'})
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Number of Games Played',
    hovermode='x unified'
)
fig.show()

In [91]:
win_rate = df.assign(
    Issac_Win_Rate = lambda x: x['Issac Wins'] / x['Game'],
    Anu_Win_Rate = lambda x: x['Anu Wins'] / x['Game']
)

fig = px.area(win_rate, x='Game', y=['Issac_Win_Rate', 'Anu_Win_Rate'], title='Win Rate Over Time')
fig.add_hline(y=0.5, line_dash='dash', line_color='black', annotation_text='50% Win Rate')
fig.show()


In [92]:
scratches = df.assign(
    Issac_Scratches = lambda x: x["Issac Shots"].str.count("S"),
    Anu_Scratches = lambda x: x["Anu Shots"].str.count("S")
)

scratches[["Issac_Scratches", "Anu_Scratches"]].describe(), scratches[["Issac_Scratches", "Anu_Scratches"]].sum()

(1      Issac_Scratches  Anu_Scratches
 count       509.000000     510.000000
 mean          1.497053       1.462745
 std           1.235344       1.127258
 min           0.000000       0.000000
 25%           1.000000       1.000000
 50%           1.000000       1.000000
 75%           2.000000       2.000000
 max           6.000000       6.000000,
 1
 Issac_Scratches    762.0
 Anu_Scratches      746.0
 dtype: float64)

In [108]:
turns = df.assign(
    Issac_Turns = df.apply(lambda x: len([i for i in str(x["Issac Shots"]).split(",") if x["Break"] == "Anagha" or i.strip()]) if pd.notna(x["Issac Shots"]) else 1 if pd.notna(x["Issac Shots"]) != pd.notna(x["Anu Shots"]) else pd.NA, axis=1),
    Anu_Turns = df.apply(lambda x: len([i for i in str(x["Anu Shots"]).split(",") if x["Break"] == "Issac" or i.strip()]) if pd.notna(x["Anu Shots"]) else 1 if pd.notna(x["Anu Shots"]) != pd.notna(x["Issac Shots"]) else pd.NA, axis=1)
)

# if any, there is an error in the formatting of the shots column
turns[turns['Issac_Turns'] != turns['Anu_Turns']]
turns["Turns"] = turns["Issac_Turns"]

In [109]:
px.histogram(turns, x="Turns", title="Turns per Game")